---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
def blight_model():
    
    trainingset = pd.read_csv('train.csv', encoding = "ISO-8859-1", low_memory=False)
    testset = pd.read_csv('test.csv', encoding="ISO-8859-1", low_memory=False)
    address = pd.read_csv('addresses.csv',encoding = "ISO-8859-1", low_memory=False)
    latlon = pd.read_csv('latlons.csv',encoding = "ISO-8859-1", low_memory=False)
    address = address.set_index('address').join(latlon.set_index('address'), how='left')
    trainingset = trainingset.set_index('ticket_id').join(address.set_index('ticket_id'))
    testset = testset.set_index('ticket_id').join(address.set_index('ticket_id'))
    modifiedtrainingset = trainingset.copy()
    modifiedtestset = testset.copy()
    ## Remove Columns from Training set which are not present in test.
    dropcolumns_trainingset = ['payment_amount', 'payment_date', 'payment_status'
                              ,'balance_due', 'collection_status', 'compliance_detail']
    modifiedtrainingset.drop(dropcolumns_trainingset, axis = 1, inplace=True)
    modifiedtrainingset = modifiedtrainingset[(modifiedtrainingset['hearing_date'].isnull() == False)]
    # Filter null valued compliance rows
    y_train = modifiedtrainingset['compliance'].dropna()
    modifiedtrainingset = modifiedtrainingset.loc[y_train.index, :]
    dropcolumns_common = ['agency_name', 'inspector_name', 'violator_name',
                       'violation_street_number', 'violation_street_name',
                       'violation_zip_code', 'mailing_address_str_number',
                       'mailing_address_str_name', 'city', 'state', 'zip_code',
                       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
                       'violation_code', 'violation_description', 'disposition','grafitti_status'
                     ]


    modifiedtrainingset.drop(['compliance'], axis=1, inplace=True)
    modifiedtrainingset.drop(dropcolumns_common, axis = 1, inplace=True)
    modifiedtestset.drop(dropcolumns_common, axis = 1, inplace=True)
    modifiedtrainingset['lat'].fillna(method='pad', inplace=True)
    modifiedtrainingset['lon'].fillna(method='pad', inplace=True)
    modifiedtestset['lat'].fillna(method='pad', inplace=True)
    modifiedtestset['lon'].fillna(method='pad', inplace=True)
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(modifiedtrainingset)
    X_test_scaled = scaler.transform(modifiedtestset)
    clf = LogisticRegression(random_state = 0).fit(X_train_scaled, y_train)
    y_predict_proba = clf.predict_proba(X_test_scaled)[:, 1]
    testset['compliance'] = y_predict_proba
    testset['compliance']
    return pd.Series(testset['compliance'])

blight_model()

ticket_id
284932    0.068076
285362    0.011475
285361    0.078914
285338    0.077157
285346    0.094657
285345    0.077262
285347    0.104611
285342    0.140608
285530    0.014953
284989    0.039065
285344    0.103999
285343    0.013879
285340    0.013967
285341    0.104601
285349    0.094689
285348    0.077289
284991    0.039066
285532    0.043972
285406    0.035511
285001    0.040482
285006    0.013707
285405    0.011431
285337    0.033462
285496    0.094428
285497    0.069534
285378    0.011371
285589    0.033854
285585    0.065153
285501    0.085721
285581    0.011330
            ...   
376367    0.020783
376366    0.060531
376362    0.058301
376363    0.064705
376365    0.020783
376364    0.060531
376228    0.066759
376265    0.060158
376286    0.126332
376320    0.064895
376314    0.064927
376327    0.144352
376385    0.132612
376435    0.231463
376370    0.144585
376434    0.088884
376459    0.088961
376478    0.000002
376473    0.064824
376484    0.059932
376482    0.034524
37